<a href="https://colab.research.google.com/github/Saifullah785/deep-learning-ai-journey/blob/main/Lecture_20_keras_hyperparameter_tuning/Lecture_20_keras_hyperparameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook Description

This notebook demonstrates the process of building and tuning a neural network model for diabetes prediction using the Pima Indians Diabetes Dataset.

The key steps covered are:

1.  **Data Loading and Exploration**: The diabetes dataset is loaded from a CSV file, and the first few rows are displayed to understand the data structure.
2.  **Feature Analysis**: The correlation of each feature with the target variable ('Outcome') is calculated to identify potential relationships.
3.  **Data Preprocessing**:
    *   Features (x) and the target variable (y) are separated.
    *   Feature scaling is performed using `StandardScaler` to standardize the range of features.
    *   The scaled features and their shape are displayed.
4.  **Data Splitting**: The dataset is split into training and testing sets using `train_test_split` for model training and evaluation.
5.  **Model Building (Initial)**: A basic sequential neural network model is created with one hidden layer and an output layer.
6.  **Hyperparameter Tuning using Keras Tuner**:
    *   **Optimizer Tuning**: Keras Tuner is used to find the best optimizer among 'adam', 'sgd', 'rmsprop', and 'adadelta' based on validation accuracy.
    *   **Number of Units Tuning**: Keras Tuner is used to find the optimal number of units in the dense layer.
    *   **Number of Layers Tuning**: Keras Tuner is used to find the best number of hidden layers in the model.
    *   **All Hyperparameters Tuning**: Keras Tuner is used to simultaneously tune the number of layers, units, activation functions, dropout rates, and the optimizer to find the best overall model configuration.
7.  **Model Training**: The best models found by Keras Tuner are trained for a specified number of epochs, and their performance on the validation data is monitored.

In [42]:
# Import necessary libraries
import numpy as np
import pandas as pd

In [43]:
# Load the diabetes dataset from Google Drive
df = pd.read_csv('/content/drive/MyDrive/diabetes.csv')

In [44]:
# Display the first 5 rows of the DataFrame
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [45]:
# Calculate the correlation of each feature with the 'Outcome' variable
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [46]:
# Separate features (x) and target variable (y)
x = df.iloc[:,0:-1].values # Features (all columns except the last one)
y = df.iloc[:,-1].values # Target variable (the last column)

In [47]:
# Import StandardScaler for feature scaling
from sklearn.preprocessing import StandardScaler
# Create a StandardScaler object
scaler = StandardScaler()

In [48]:
# Fit and transform the features using the scaler
x = scaler.fit_transform(x)

In [49]:
# Display the scaled features
x

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [50]:
# Display the shape of the scaled features array
x.shape

(768, 8)

In [51]:
# Import train_test_split for splitting data
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets (80% train, 20% test)
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=1)

In [52]:
# Import necessary libraries for building the neural network
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [53]:
# Create a Sequential model
model = Sequential()
# Add a dense layer with 32 units and 'relu' activation, specifying input dimension
model.add(Dense(32, activation='relu', input_dim=8))
# Add the output layer with 1 unit and 'sigmoid' activation for binary classification
model.add(Dense(1, activation='sigmoid'))

# Compile the model with Adam optimizer, binary crossentropy loss, and accuracy metric
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics = ['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [54]:
# Train the model
# x_train: training features
# y_train: training labels
# batch_size: number of samples per gradient update
# epochs: number of times to iterate over the entire training dataset
# validation_data: data to evaluate the model on after each epoch
model.fit(x_train, y_train, batch_size=32, epochs=100, validation_data=(x_test, y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.4540 - loss: 0.7238 - val_accuracy: 0.5714 - val_loss: 0.6852
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5786 - loss: 0.6782 - val_accuracy: 0.6818 - val_loss: 0.6384
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6995 - loss: 0.6192 - val_accuracy: 0.7468 - val_loss: 0.6021
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7152 - loss: 0.5924 - val_accuracy: 0.7597 - val_loss: 0.5771
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7332 - loss: 0.5683 - val_accuracy: 0.7468 - val_loss: 0.5548
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7330 - loss: 0.5533 - val_accuracy: 0.7532 - val_loss: 0.5379
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7435 - loss: 0.5241 - val_accuracy: 0.7597 - val_loss: 0.5232
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7125 - loss: 0.5355 - val_accuracy: 0.7597 - 



---


# **1.how to select appropriate optimizer**
# **2.No. of nodes in a layer**
# **3.how to select no. of layers**
# **4.All in all one model**

In [55]:
# Install Keras Tuner
!pip install -U keras-tuner

## **1.how to select appropriate optimizer**

In [56]:
# Import keras_tuner
import keras_tuner as kt

In [57]:
# Define a function to build the model for Keras Tuner
def build_model(hp):
  model = Sequential()
  # Add a dense layer with 32 units and 'relu' activation, specifying input dimension
  model.add(Dense(32, activation='relu', input_dim=8))
  # Add the output layer with 1 unit and 'sigmoid' activation for binary classification
  model.add(Dense(1, activation='sigmoid'))

  # Define the optimizer to be tuned
  optimizer=hp.Choice('optimizer', values = ['adam', 'sgd', 'rmsprop', 'adadelta' ])

  # Compile the model with the chosen optimizer, binary crossentropy loss, and accuracy metric
  model.compile(optimizer= optimizer, loss= 'binary_crossentropy', metrics=['accuracy'])
  return model

In [58]:
# Initialize the RandomSearch tuner to find the best optimizer
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy', # Objective to optimize
                        max_trials=5) # Number of trials to run

Reloading Tuner from ./untitled_project/tuner0.json


In [59]:
# Perform the hyperparameter search for the best optimizer
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test,y_test))

In [60]:
# Get the best hyperparameters found by the tuner
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [61]:
# Get the best model found by the tuner
model =tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [62]:
# Display the summary of the best model's architecture
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [63]:
# Train the best model for more epochs, starting from the initial epoch
model.fit(x_train, y_train, batch_size=32, epochs=100, initial_epoch=6,validation_data=(x_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 55ms/step - accuracy: 0.7493 - loss: 0.4970 - val_accuracy: 0.7727 - val_loss: 0.5214
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.7395 - loss: 0.4997 - val_accuracy: 0.7727 - val_loss: 0.5094
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7347 - loss: 0.5144 - val_accuracy: 0.7662 - val_loss: 0.5019
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7408 - loss: 0.5091 - val_accuracy: 0.7597 - val_loss: 0.4970
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7971 - loss: 0.4520 - val_accuracy: 0.7727 - val_loss: 0.4934
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7900 - loss: 0.4550 - val_accuracy: 0.7792 - val_loss: 0.4901
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7857 - loss: 0.4520 - val_accuracy: 0.7857 - val_loss: 0.4877
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7635 - loss: 0.4922 - val_accuracy

## **2.No. of nodes in a layer**

In [64]:
# Define a function to build the model for Keras Tuner to tune the number of units in a layer
def build_model(hp):
  model = Sequential()

  # Tune the number of units in the dense layer
  units = hp.Int('units', min_value=8, max_value=128)
  model.add(Dense(units=units,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  # Compile the model with the rmsprop optimizer (found as best in previous tuning)
  model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [65]:
# Initialize the RandomSearch tuner to find the best number of units
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy', # Objective to optimize
                        max_trials=5, # Number of trials to run
                        directory='mydir', # Directory to save the results
                        project_name='srk') # Project name

Reloading Tuner from mydir/srk/tuner0.json


In [66]:
# Perform the hyperparameter search for the best number of units
tuner.search(x_train, y_train,epochs =5, validation_data=(x_test,y_test))

In [67]:
# Get the best hyperparameters found by the tuner for the number of units
tuner.get_best_hyperparameters()[0].values

{'units': 58}

In [68]:
# Get the best model found by the tuner for the number of units
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [82]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 56)             │           504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 56)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 72)             │         4,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 72)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            73 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,045 (54.87 KB)

 Trainable params: 4,681 (18.29 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 9,364 (36.58 KB)

In [69]:
# Train the best model for more epochs, starting from the initial epoch
model.fit(x_train, y_train, batch_size=32, epochs=100, initial_epoch=6, validation_data=(x_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.7396 - loss: 0.5273 - val_accuracy: 0.7922 - val_loss: 0.4941
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7385 - loss: 0.5256 - val_accuracy: 0.7792 - val_loss: 0.4813
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7651 - loss: 0.4968 - val_accuracy: 0.7857 - val_loss: 0.4716
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7246 - loss: 0.5124 - val_accuracy: 0.7922 - val_loss: 0.4645
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7792 - loss: 0.4874 - val_accuracy: 0.7922 - val_loss: 0.4604
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7859 - loss: 0.4619 - val_accuracy: 0.7987 - val_loss: 0.4557
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7739 - loss: 0.4645 - val_accuracy: 0.7922 - val_loss: 0.4533
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7915 - loss: 0.4400 - val_accuracy: 0.79

## **3.how to select no. of layers**

In [70]:
# Define a function to build the model for Keras Tuner to tune the number of layers
def build_model(hp):
  model = Sequential()
  # Add the first dense layer
  model.add(Dense(32, activation='relu',input_dim=8))

  # Tune the number of hidden layers
  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
    # Add dense layers with 8 units and 'relu' activation
    model.add(Dense(units=8, activation='relu'))

  # Add the output layer
  model.add(Dense(1, activation='sigmoid'))

  # Compile the model with the rmsprop optimizer and binary crossentropy loss
  model.compile(optimizer = 'rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [71]:
# Initialize the RandomSearch tuner to find the best number of layers
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy', # Objective to optimize
                        max_trials=3, # Number of trials to run
                        directory='mydir', # Directory to save the results
                        project_name='num_layers') # Project name

Reloading Tuner from mydir/num_layers/tuner0.json


In [72]:
# Perform the hyperparameter search for the best number of layers
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test,y_test))

In [73]:
# Get the best hyperparameters found by the tuner for the number of layers
tuner.get_best_hyperparameters()[0].values

{'num_layers': 4}

In [74]:
# Get the best model found by the tuner for the number of layers
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [75]:
# Train the best model for more epochs, starting from the initial epoch
model.fit(x_train, y_train, epochs=100, initial_epoch=6, validation_data=(x_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.6778 - loss: 0.6683 - val_accuracy: 0.6494 - val_loss: 0.6588
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6709 - loss: 0.6484 - val_accuracy: 0.6558 - val_loss: 0.6343
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6531 - loss: 0.6210 - val_accuracy: 0.6688 - val_loss: 0.6087
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6745 - loss: 0.5940 - val_accuracy: 0.6688 - val_loss: 0.5821
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6769 - loss: 0.5614 - val_accuracy: 0.6753 - val_loss: 0.5586
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6927 - loss: 0.5291 - val_accuracy: 0.7078 - val_loss: 0.5411
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7259 - loss: 0.5067 - val_accuracy: 0.7143 - val_loss: 0.5269
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7708 - loss: 0.4867 - val_accuracy: 0.75

## **4.All in all one model**

In [76]:
# Define a function to build the model for Keras Tuner to tune multiple hyperparameters
def build_model(hp):
  model = Sequential()

  counter = 0

  # Tune the number of hidden layers
  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):

    if counter == 0:
      # Tune units, activation, and dropout for the first hidden layer
      model.add(
          Dense(
              hp.Int('units' + str(i), min_value=8, max_value=128,step=8),
              activation= hp.Choice('activation' + str(i) , values=['relu','tanh','sigmoid']),
              input_dim=8
              )
          )
      model.add(Dropout(hp.Choice('dropout' + str(i), values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    else:
      # Tune units, activation, and dropout for subsequent hidden layers
      model.add(
          Dense(
                hp.Int('units' + str(i), min_value=8, max_value=128,step=8),
                activation= hp.Choice('activation' + str(i) , values=['relu','tanh','sigmoid'])
                )
            )
      model.add(Dropout(hp.Choice('dropout' + str(i), values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    counter += 1 # Increment counter after adding a layer

  # Add the output layer
  model.add(Dense(1, activation='sigmoid'))

  # Tune the optimizer
  model.compile(optimizer=hp.Choice('optimizer', values = ['adam', 'sgd', 'rmsprop', 'adadelta' ]),
                loss='binary_crossentropy',
                metrics=['accuracy'])
  return model

In [77]:
# Initialize the RandomSearch tuner to tune all hyperparameters
tuner = kt.RandomSearch(build_model,
                       objective='val_accuracy', # Objective to optimize
                       max_trials=3, # Number of trials to run
                       directory='mydir', # Directory to save the results
                       project_name='final1') # Project name

Reloading Tuner from mydir/final1/tuner0.json


In [78]:
# Perform the hyperparameter search
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test,y_test))

In [79]:
# Get the best hyperparameters found by the tuner
tuner.get_best_hyperparameters()[0].values

{'num_layers': 2,
 'units0': 56,
 'activation0': 'tanh',
 'dropout0': 0.2,
 'optimizer': 'adam',
 'units1': 72,
 'activation1': 'sigmoid',
 'dropout1': 0.6,
 'units2': 80,
 'activation2': 'relu',
 'dropout2': 0.4,
 'units3': 8,
 'activation3': 'tanh',
 'dropout3': 0.7,
 'units4': 24,
 'activation4': 'tanh',
 'dropout4': 0.3,
 'units5': 112,
 'activation5': 'sigmoid',
 'dropout5': 0.6}

In [80]:
# Get the best model found by the tuner
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [81]:
# Train the best model for more epochs, starting from the initial epoch
model.fit(x_train, y_train, epochs=100, initial_epoch=6, validation_data=(x_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 50ms/step - accuracy: 0.7117 - loss: 0.5657 - val_accuracy: 0.7922 - val_loss: 0.4769
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.7116 - loss: 0.5556 - val_accuracy: 0.7792 - val_loss: 0.4710
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.7678 - loss: 0.4992 - val_accuracy: 0.7727 - val_loss: 0.4681
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.7567 - loss: 0.5044 - val_accuracy: 0.7662 - val_loss: 0.4659
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.7334 - loss: 0.5411 - val_accuracy: 0.7662 - val_loss: 0.4645
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7737 - loss: 0.5032 - val_accuracy: 0.7792 - val_loss: 0.4649
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7499 - loss: 0.5239 - val_accuracy: 0.7727 - val_loss: 0.4617
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7534 - loss: 0.5227 - val_accuracy